Importing Data

In [2]:
pip install datasets


In [11]:
import numpy as np
from sklearn.utils import shuffle
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from datasets import load_dataset
from collections import Counter
#from __future__ import print_function
import collections
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import re as re
import nltk
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split
from torch.nn import functional as F
from torch.utils.data import TensorDataset, random_split
from sklearn import preprocessing
from sklearn.metrics import precision_recall_fscore_support

Loading data

In [3]:
dataset = load_dataset("dbpedia_14")
batch_size = 64
df=pd.DataFrame(dataset['train'])
df2=pd.DataFrame(dataset['test'])
df['content']=df['content'].str.lower()
df2['content']=df2['content'].str.lower()
####Shuffle 

Reusing dataset d_bpedia14 (/root/.cache/huggingface/datasets/d_bpedia14/dbpedia_14/2.0.0/7f0577ea0f4397b6b89bfe5c5f2c6b1b420990a1fc5e8538c7ab4ec40e46fa3e)


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
### Count 50000 most frequent words
###make word to ind and ind to word dictionaries
nltk.download("popular")
stop = nltk.corpus.stopwords.words('english')
df['text_without_stopwords'] = df['content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

def search_words(text):
    result = re.findall(r'\b[^\d\W]+\b', text)
    return result
arr=df['text_without_stopwords'].apply(lambda x : search_words(x))
arr_test=df2['content'].apply(lambda x: search_words(x))
data_train_text=df.iloc[:,2]
data_train_label=df.iloc[:,0]

wordcount = {}
# To eliminate duplicates, remember to split by punctuation, and use case demiliters.
for i in range(arr.shape[0]):
  for word in arr[i]:
    word = word.replace(".","")
    word = word.replace(",","")
    word = word.replace(":","")
    word = word.replace("\"","")
    word = word.replace("!","")
    word = word.replace("â€œ","")
    word = word.replace("â€˜","")
    word = word.replace("*","")
    #if word not in stopwords:
    if word not in wordcount:
      wordcount[word] = 1
    else:
       wordcount[word] += 1
# Print most common word
cnt=50000
word_counter = collections.Counter(wordcount)
wordsmostcommon=word_counter.most_common(cnt)
word2ind={}
ind2word=[]
count=0
for word in wordsmostcommon:
  word2ind[word[0]]=count
  count=count+1

for word in word2ind:
  ind2word.append(word)

ind2word.append('efkhv')



[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

In [7]:
###convert string of content to numbers
data_train_nos=[]
for line in arr:
  line_words=[]
  for words in line:
    line_words.append(word2ind.get(words,0))
  data_train_nos.append(line_words)

  mx=-1
for i in data_train_nos:
    if(mx<len(i)):
        mx=len(i)
print(mx)

for sentence in data_train_nos:
    while(len(sentence)<mx):
      sentence.append(50000)

1104


In [8]:
###convert string of content to numbers
data_test_nos=[]
for line in arr_test:
  line_words=[]
  for words in line:
    line_words.append(word2ind.get(words,0))
  data_test_nos.append(line_words)

  mx=-1
for i in data_test_nos:
    if(mx<len(i)):
        mx=len(i)
print(mx)

for sentence in data_test_nos:
    while(len(sentence)<mx):
      sentence.append(50000)

384


In [64]:
tensordata_train=TensorDataset(torch.Tensor(data_train_nos[0:20000]),torch.tensor(df['label'].iloc[0:20000]))
tensordata_test=TensorDataset(torch.Tensor(data_test_nos[0:20000]),torch.tensor(df2['label'].iloc[0:20000]))


In [65]:
 def collate_func(data):
        examples, labels = zip(*data)
        labels = torch.tensor(labels)
        dataword=[]
        #print(examples)
        #print(labels)
        for line in examples:
          #print(line)
          lines_trans=[]
          for words in line:
            lines_trans.append(ind2word[int(words)])
          dataword.append(' '.join(lines_trans))

       # print(type(dataword))
       # print(dataword)
        vectorizer = CountVectorizer(vocabulary=word2ind)
        bag = vectorizer.fit_transform(dataword) 
        features_unnormal=bag.toarray()
        print(type(features_unnormal))
        print(features_unnormal.shape)
        print(type(features_unnormal[0]))
        #features_unnormal=torch.tensor(features_unnormal)
        features = preprocessing.normalize( features_unnormal,axis=0)
        features=torch.tensor(features)
       # print(type(features))
        return features.float(), labels.long()

In [66]:
batchwise_train=DataLoader(tensordata_train, batch_size=128,collate_fn=collate_func)
batchwise_test=DataLoader(tensordata_test, batch_size=128,collate_fn=collate_func)
generator = iter(batchwise_test)
print(torch.sum(next(generator)[0]))

<class 'numpy.ndarray'>
(128, 50000)
<class 'numpy.ndarray'>
tensor(2360.2405)


In [69]:
import torch.nn as nn

input_size = 50000
num_classes = 14

model = nn.Linear(input_size, num_classes)

In [70]:
    def accuracy(outputs, labels):
      _, preds = torch.max(outputs, dim=1)
      return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [71]:
class dbpedia(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, num_classes)
        
    def forward(self, xb):
        # xb = xb.reshape(-1, 7)
        out = self.linear(xb)
        return out
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss


    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)  
        _, preds = torch.max(out, dim=1)                 # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)
        prec=precision_recall_fscore_support(labels.cpu().detach().numpy(), preds.cpu().detach().numpy(), average='macro')         # Calculate accuracy
        return {'val_loss': loss, 'val_acc': acc, 'val_prec':torch.tensor(prec[0]), 'val_recall':torch.tensor(prec[1]), 'val_fscore':torch.tensor(prec[2])}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        batch_prec = [x['val_prec'] for x in outputs]
        epoch_prec = torch.stack(batch_prec).mean()
        batch_recall = [x['val_recall'] for x in outputs]
        epoch_recall = torch.stack(batch_recall).mean()
        batch_fscore = [x['val_fscore'] for x in outputs]
        epoch_fscore = torch.stack(batch_fscore).mean()
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item(), 'val_prec':epoch_prec.item(), 'val_recall':epoch_recall.item(), 'val_fscore': epoch_fscore.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}, val_prec: {:.4f}, val_recall: {:.4f}, val_fscore: {:.4f}".format(epoch, result['val_loss'], result['val_acc'],result['val_prec'],result['val_recall'],result['val_fscore']))
    
model = dbpedia()

In [72]:
def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    optimizer = opt_func(model.parameters(), lr)
    history = [] # for recording epoch-wise results
    
    for epoch in range(epochs):
        
        # Training Phase 
        for batch in train_loader:
            #print(batch[0])
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result)
        history.append(result)

    return history


def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

In [ ]:
history1 = fit(1, 0.001, model, batchwise_train, batchwise_test)

<class 'numpy.ndarray'>
(128, 50000)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(128, 50000)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(128, 50000)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(128, 50000)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(128, 50000)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(128, 50000)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(128, 50000)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(128, 50000)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(128, 50000)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(128, 50000)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(128, 50000)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(128, 50000)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(128, 50000)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(128, 50000)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(128, 50000)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(128, 50000)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<class 'numpy.ndarray'>
(128, 50000)
<class 'numpy.ndarray'>


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<class 'numpy.ndarray'>
(128, 50000)
<class 'numpy.ndarray'>


...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<class 'numpy.ndarray'>
(128, 50000)
<class 'numpy.ndarray'>


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<class 'numpy.ndarray'>
(128, 50000)
<class 'numpy.ndarray'>


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
